In [2]:
import nltk
import random
import numpy as np

from bs4 import BeautifulSoup

In [8]:
nltk.download('punkt')      # for nltk tokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!wget https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/nlp_class/electronics/positive.review

--2021-05-14 15:51:10--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/nlp_class/electronics/positive.review
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1105010 (1.1M) [text/plain]
Saving to: ‘positive.review’

positive.review     100%[===================>]   1.05M  --.-KB/s    in 0.07s   

2021-05-14 15:51:11 (14.5 MB/s) - ‘positive.review’ saved [1105010/1105010]



In [6]:
with open('positive.review') as f:
    positive_reviews = BeautifulSoup(f.read()).findAll('review_text')

In [10]:
# `trigrams` saves the probability of middle word based on neighbouring words, i.e. trigram[('w1', 'w3')] = {w2_1, w2_2, ...}
trigrams = {}

for review in positive_reviews:
    s = review.text.lower()
    tokens = nltk.tokenize.word_tokenize(s)
    for i in range(len(tokens) - 2):
        k = (tokens[i], tokens[i+2])
        if k not in trigrams:
            trigrams[k] = []
        trigrams[k].append(tokens[i+1])

# trigram[('w1', 'w3')] = {w2_1 : p2_1, w2_2 : p2_2, ...}

In [17]:
trigrams_prob = {}              # trigram_prob[('w1', 'w3')] = {w2_1 : p2_1, w2_2 : p2_2, ...}
for k, words in trigrams.items():
    if len(set(words)) > 1:     # ignore trigram with only one possibility
        d = {}
        n = 0   # total # words in a trigram
        for w in words:
            if w not in d:
                d[w] = 0
            d[w] += 1
            n += 1
        for w, c in d.items():
            d[w] = c / n
        trigrams_prob[k] = d

In [24]:
def random_sample(d):       # randomly pick a word to change / spin
    r = random.random()
    cumulative = 0
    for w, p in d.items():
        cumulative += p 
        if r < cumulative:
            return w

In [27]:
review = random.choice(positive_reviews)        # randomly choose a review to spin
s = review.text.lower()
print("Ori : ", s.replace("\n", " "))
tokens = nltk.tokenize.word_tokenize(s)
for i in range(len(tokens) - 2):
    if random.random() < 0.2 :   # consider to spin only 20% of the tokens
        k = (tokens[i], tokens[i+2])
        if k in trigrams_prob:
            w = random_sample(trigrams_prob[k])
            tokens[i+1] = w
spun = " ".join(tokens)
print("Spun : ", spun)

# word is replaced based on word occurrence in trigram... so sometimes the context might not make sense 

# Further Effort:
    # Explore more grams
    # add more data samples, e.g. from wikipedia
    # try modeling with past words instead of surrounding ones
    # match POS (part of speech) in grams so that it's more grammatically correct 
    # use word2vec or GloVe from deep learning to replace word closer to the original one in word vectors
    # apply RNN perhaps

Ori :   panasonic makes good quality products and this is no exception.  easy to use, very compact, no interference with cordless usage 
Spun :  panasonic makes good quality products and this is no exception . easy to move , very compact , no interference with cordless usage
